In [4]:
import os
is_kaggle=os.environ.get("KAGGLE_KERNEL_RUN_TYPE","")


In [6]:
from pathlib import Path
cred_path=cred_path = Path('~/.kaggle/kaggle.json').expanduser()
if not cred_path.exists():
    cred_path.parent.mkdir(exist_ok=True)
    cred_path.write_text(creds)
    cred_path.chmod(0o600)

In [9]:
path=Path('us-patent-phrase-to-phrase-matching')
# ! kaggle competitions download -c us-patent-phrase-to-phrase-matching
# ! unzip us-patent-phrase-to-phrase-matching.zip

100%|████████████████████████████████████████| 682k/682k [00:00<00:00, 4.19MB/s]
100%|████████████████████████████████████████| 682k/682k [00:00<00:00, 4.13MB/s]


In [10]:
# if not is_kaggle and not path.exists():
#     import zipfile,kaggle
#     kaggle.api.dataset_download_cli(str(path))
#     zipfile.ZipFile(f'{path}.zip').extractall(path)

In [11]:
!ls

getting-started-with-nlp-for-absolute-beginners.ipynb
intro_nlp.ipynb
sample_submission.csv
test.csv
train.csv


In [1]:
import pandas as pd
import numpy as np 
import matplotlib.pyplot as plt

In [2]:
df=pd.read_csv('train.csv')

In [3]:
df.head(10)

,id,anchor,target,context,score
0,37d61fd2272659b1,abatement,abatement of pollution,A47,0.50
1,7b9652b17b68b7a4,abatement,act of abating,A47,0.75
2,36d72442aefd8232,abatement,active catalyst,A47,0.25
3,5296b0c19e1ce60e,abatement,eliminating process,A47,0.50
4,54c1e3b9184cb5b6,abatement,forest region,A47,0.00
5,067203128142739c,abatement,greenhouse gases,A47,0.25
6,061d17f04be2d1cf,abatement,increased rate,A47,0.25
7,e1f44e48399a2027,abatement,measurement level,A47,0.25
8,0a425937a3e86d10,abatement,minimising sounds,A47,0.50
9,ef2d4c2e6bbb208d,abatement,mixing core materials,A47,0.25


In [4]:
df.describe(include='object')

,id,anchor,target,context
count,36473,36473,36473,36473
unique,36473,733,29340,106
top,37d61fd2272659b1,component composite coating,composition,H01
freq,1,152,24,2186


In [5]:
df['input']='TEXT1:'+df.context+';'+'TEXT2:'+df.target+';'+'TEXT3:'+df.anchor+';'

In [6]:
from datasets import Dataset,DatasetDict
ds=Dataset.from_pandas(df)

In [7]:
ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'score', 'input'],
    num_rows: 36473
})

In [8]:
model_nm='microsoft/deberta-v3-small'

In [9]:
from transformers import AutoModelForSequenceClassification,AutoTokenizer
tokz = AutoTokenizer.from_pretrained(model_nm)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
/Volumes/TAPPS/DS_APPS/anaconda3/envs/tensorflow/lib/python3.9/site-packages/transformers/convert_slow_tokenizer.py:454: UserWarning: The sentencepiece tokenizer that you are converting to a fast tokenizer uses the byte fallback option which is not implemented in the fast tokenizers. In practice this means that the fast version of the tokenizer can produce unknown tokens whereas the sentencepiece version would have converted these unknown tokens into a sequence of byte tokens matching the original piece of text.
  warnings.warn(
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [10]:
def tok_fun(x):
    return tokz(x['input'])

In [11]:
tok_ds=ds.map(tok_fun, batched=True)

Map:   0%|          | 0/36473 [00:00<?, ? examples/s]

In [18]:
row=tok_ds[0]
tok_ds=tok_ds.rename_columns({'score':'labels'})

In [19]:
tok_ds

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36473
})

In [20]:
eval_df=pd.read_csv('test.csv')

In [22]:
eval_df.describe()

,id,anchor,target,context
count,36,36,36,36
unique,36,34,36,29
top,4112d61851461f60,el display,inorganic photoconductor drum,G02
freq,1,2,1,3


In [24]:
dds=tok_ds.train_test_split(test_size=0.25,seed=42)
dds

DatasetDict({
    train: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 27354
    })
    test: Dataset({
        features: ['id', 'anchor', 'target', 'context', 'labels', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 9119
    })
})

In [26]:
eval_df['input']='TEXT1:'+eval_df.context+';'+'TEXT2:'+eval_df.target+';'+'TEXT3:'+eval_df.anchor+';'
eval_ds=Dataset.from_pandas(eval_df).map(tok_fun,batched=True)
eval_ds

Map:   0%|          | 0/36 [00:00<?, ? examples/s]

Dataset({
    features: ['id', 'anchor', 'target', 'context', 'input', 'input_ids', 'token_type_ids', 'attention_mask'],
    num_rows: 36
})

In [28]:
from transformers import TrainingArguments,Trainer

In [30]:
bs=128
epochs=4
lr=8e-5

In [33]:
args = TrainingArguments('outputs', learning_rate=lr, warmup_ratio=0.1, lr_scheduler_type='cosine', fp16=True,
    evaluation_strategy="epoch", per_device_train_batch_size=bs, per_device_eval_batch_size=bs*2,
    num_train_epochs=epochs, weight_decay=0.01, report_to='none')

NameError: name 'PartialState' is not defined